In [ ]:
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer

from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

import spacy
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import re

%pip install emoji
import emoji


from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBRegressor
from sklearn import svm


%pip install optuna
import optuna
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold#, GridSearchCV

In [ ]:
nlp = spacy.load("en_core_web_sm")

df = pd.read_csv(df_url, sep='\t', header=0)
df = df.drop(columns=[col for col in df.columns if "Unnamed" in col] + ["tweet_id"])
df["text"] = df["text"].astype(str)

In [ ]:
class EmojiExtractor(BaseEstimator, TransformerMixin):
	def fit(self, X, y=None):
		return self

	def transform(self, X):
		X = X.copy()

		def count_emojis(text):
			return sum(1 for char in text if char in emoji.EMOJI_DATA)

		def replace_emojis(text):
			return emoji.demojize(text, delimiters=(" ", " "))

		X["emoji_count"] = X["text"].apply(count_emojis)
		X["text"] = X["text"].apply(replace_emojis)

		return X

class LinkExtractor(BaseEstimator, TransformerMixin):
	def fit(self, X, y=None):
		return self

	def transform(self, X):
		X = X.copy()

		def count_links(text):
			return len(re.findall(r'https?:\/\/.*[\r\n]*', text))

		def remove_links(text):
			return re.sub(r'https?:\/\/\S+', '[url]', text)

		X["link_count"] = X["text"].apply(count_links)
		X["text"] = X["text"].apply(remove_links)

		return X
	
class TextPreprocessor(BaseEstimator, TransformerMixin):
	def fit(self, X, y=None):
		return self

	def transform(self, X):
		X = X.copy()
		def preprocess(text):
			doc = nlp(text.lower())
			return " ".join(token.lemma_ for token in doc if not token.is_stop and not token.is_punct)

		X["text"] = X["text"].apply(preprocess)
		return X

## Define the pipeline for the model

In [ ]:
X = df.copy().drop(columns=["scientific_claim", "scientific_reference", "scientific_context"])
y = X.pop("science_related")

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=5, test_size=0.2)
pd.set_option('display.max_colwidth', 110)
X_train.head()

In [ ]:
def display_result(predicted_y, true_y ):
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))
    cm = confusion_matrix(true_y, predicted_y)
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=["Non science related", "Science related"],
            yticklabels=["Non science related", "Science related"], ax=axes[0])

    axes[0].set_xlabel("Predicted Label")
    axes[0].set_ylabel("True Label")
    axes[0].set_title("Confusion Matrix")

    target_names = ['Non science related', 'Science related']
    class_report = classification_report(true_y, predicted_y, labels=[0, 1], target_names=target_names)

    axes[1].text(0, 0.5, class_report, fontsize=12, family='monospace')
    axes[1].axis("off")
    axes[1].set_title("Classification Report")

    plt.tight_layout()
    plt.show()

In [ ]:
column_transformer = ColumnTransformer([
    ("tfidf", TfidfVectorizer(stop_words='english', ngram_range=(1,4)), "text"),
    ("pass_features", "passthrough", ["emoji_count", "link_count"])
])


def getPipeline(model):
    return Pipeline([
        ("emoji_processing", EmojiExtractor()),
        ("link_processing", LinkExtractor()),
        ("text_processing", TextPreprocessor()),
        ("feature_vectorizer", column_transformer),
        ("classifier", model)
        #("classifier", svm.SVC(kernel='linear', class_weight='balanced'))
        #("classifier", RandomForestClassifier(random_state = 1))
        #("classifier", XGBRegressor( random_state=1))
    ])

In [ ]:
def objective(trial):
  C = trial.suggest_loguniform('C', 1, 10.0)

  kernel = trial.suggest_categorical('kernel', ['linear', 'rbf'])
  gamma = trial.suggest_loguniform('gamma', 7, 10.0) if kernel == 'rbf' else 'scale'
  model = svm.SVC(C=C, kernel=kernel, gamma=gamma, random_state=1, class_weight='balanced')

  pipeline = getPipeline(model)

  kfold = KFold(n_splits=3, random_state=1, shuffle=True)
  score = cross_val_score(pipeline, X=X_train, y=y_train,  cv=kfold, scoring='f1_macro').mean()

  return score

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=4)

In [ ]:
print("Meilleurs hyperparamètres :")
print(study.best_params)
print(f"Meilleur score : {study.best_value:.3f}")


In [ ]:
model = svm. SVC(
  C=study.best_params['C'],
  kernel=study.best_params['kernel'],
  random_state=1,
  class_weight='balanced'
  )
pipeline = getPipeline(model)
pipeline.fit(X_train, y_train)

In [ ]:
predictions = pipeline.predict(X_test)
display_result(predictions, y_test )

In [ ]:
def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 300)
    max_depth = trial.suggest_int('max_depth', 4, 20)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 4)

    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        random_state=1,
        class_weight='balanced'
    )

    pipeline = getPipeline(model)

    kfold = KFold(n_splits=3, random_state=1, shuffle=True)
    score = cross_val_score(pipeline, X=X_train, y=y_train, cv=kfold, scoring='f1_macro').mean()

    return score

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=6)

In [ ]:
model = RandomForestClassifier(
        n_estimators=study.best_params['n_estimators'],
        max_depth=study.best_params['max_depth'],
        min_samples_split=study.best_params['min_samples_split'],
        min_samples_leaf=study.best_params['min_samples_leaf'],
        random_state=1,
        class_weight='balanced'
    )
pipeline = getPipeline(model)
pipeline.fit(X_train, y_train)

In [ ]:
predictions = pipeline.predict(X_test)
display_result(predictions, y_test )

In [ ]:
misclassified_indices = np.where(predictions != y_test)[0]

# Récupérer les tweets mal classifiés
misclassified_tweets = X_test.iloc[misclassified_indices]['text'].tolist()
misclassified_true = y_test.iloc[misclassified_indices].tolist()
misclassified_pred = predictions[misclassified_indices].tolist()

misclassified_df = pd.DataFrame({
    'tweet': misclassified_tweets,
    'true_label': misclassified_true,
    'predicted_label': misclassified_pred
})
misclassified_df.head(20)

In [ ]:
tfidf_vectorizer = pipeline.named_steps['feature_vectorizer'].named_transformers_['tfidf']

feature_names = tfidf_vectorizer.get_feature_names_out()
tfidf_matrix = tfidf_vectorizer.transform(X['text'])
feature_scores = np.asarray(tfidf_matrix.sum(axis=0)).ravel()

feature_df = pd.DataFrame({'word': feature_names, 'tfidf_score': feature_scores})

feature_df.sort_values(by='tfidf_score', ascending=False).head(20)
